In [2]:
!nvidia-smi

Sun Jun 13 10:39:52 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Intro to NLP Fundamentals in tf 

nlp has the goal of deriving the info out of the natural language ( could be text or speech ) sequences .

another common term for NLP problems is sequence to sequence 

## checking for gpu

In [3]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-a779d942-2a49-1dd5-9b4b-2a1127dfbe05)


# Getting helper function 

In [4]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

# importing the series of helper function 
from helper_functions import unzip_data, create_tensorboard_callback , plot_loss_curves,compare_historys

--2021-06-13 10:39:57--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2021-06-13 10:39:57 (71.8 MB/s) - ‘helper_functions.py’ saved [10246/10246]



# Getting the text dataset 

the dataset we're going to use is the kaggle intro to NLP dataset ( text sample of tweets labeled as the disaster or not disaster ). 

In [5]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2021-06-13 10:39:59--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.26.128, 172.217.193.128, 172.217.204.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.26.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.007s  

2021-06-13 10:39:59 (88.2 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [6]:
unzip_data('nlp_getting_started.zip')

## Importing the required libraries 

In [7]:
import pandas as pd 
import numpy as np 
import tensorflow as tf 
import matplotlib.pyplot as plt 
import random
from sklearn.model_selection import train_test_split


# becoming one with the data 


In [8]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
sample_submission = pd.read_csv('sample_submission.csv')

In [9]:
train_data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [10]:
train_data.isna().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [11]:
train_df_shuffled = train_data.sample(frac = 1, random_state= 42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [12]:
train_data.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [13]:
# how many no of samples 
len(train_data),len(test_data)

(7613, 3263)

In [14]:
# let's visualize the random training sample 
random_index = random.randint(0,len(train_data)-5) # creating the ranodm indexes not higer 
for row in train_df_shuffled[['text','target']][random_index : random_index + 5].itertuples():
  _,text,target = row
  print(f'target: {target} ',"real disaster" if target > 0 else "not real disaster")
  print(f'text: {text}\n')

target: 1  real disaster
text: Police: Teenagers arrested for curfew violations in Evanston were riding stolen bicycles: Two teenagers taken into cus...  #Chicago #news

target: 0  not real disaster
text: Finna storm. Fuck my back boutta start hurting like a mf ??????

target: 0  not real disaster
text: Most of us ddnt get this English  RT @ReIgN_CoCo: The World Is Inundated With Ostentatious People Stay Woke!

target: 1  real disaster
text: PolicyLab is at @CECANF's last public hearing in NYC today and tomorrow to address child abuse and neglect fatalities http://t.co/n2cY3Z4TPB

target: 1  real disaster
text: Twelve feared killed in Pakistani air ambulance helicopter crash http://t.co/Xum8YLcb4Q



### splitting the data in training and validation sets

In [15]:
# using the train test split to split the train data into training and the validation steps 
train_scentences, val_scentences, train_labels, val_labels = train_test_split(train_df_shuffled['text'].to_numpy(),
                                                                              train_df_shuffled['target'].to_numpy(),
                                                                              test_size = 0.1, # 10 % of training data for validation split 
                                                                              random_state = 42)

In [16]:
len(train_scentences),len(train_labels),len(val_scentences),len(val_labels)

(6851, 6851, 762, 762)

In [17]:
# looking at the first 10 examples 
train_scentences[:10],train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 

## converting the text data into the numbers 


In [18]:
# toeknization : token level tokaization ( word level tokenization ) direct mapping from word to no 
# other form is like : character to number 
# creating the embedding every word turned in vector we can determine the dimension of the vector 


In [19]:
# doing the text vectorizatoin 
# using teh default text vectorization 

text_vectorizer = tf.keras.layers.experimental.preprocessing.TextVectorization(max_tokens=None, # automatically adjust the nubmer of vocab contain in vocab
                                                                               standardize = 'lower_and_strip_punctuation',
                                                                               split ='whitespace',
                                                                               ngrams = None ,# create groups of N words 
                                                                               output_mode = 'int',
                                                                               output_sequence_length = 50,
                                                                               pad_to_max_tokens = True)

In [20]:
len(train_scentences[0].split())

7

In [21]:
# fiding the average numbeer of words in training tweets 
round(sum([len(i.split()) for i in train_scentences])/len(train_scentences))

15

In [22]:
#setting up the text vectorization variables 
max_vocab_length= 10000
max_length = 15 # max lengths of our sequences will be 
text_vectorizer = tf.keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens = max_vocab_length,
    output_mode = 'int',
    output_sequence_length = max_length
)

In [23]:
#fitting the text vectorizer to the training state 
text_vectorizer.adapt(train_scentences)

In [24]:
sample_scen = 'saturday night is amazing there is the moon lightning up your head the million colors of hazels golden and red '
text_vectorizer([sample_scen])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[1601,  285,    9, 1720,   74,    9,    2, 2038,  300,   27,   33,
         331,    2, 1272,    1]])>

In [25]:
# chossing random scentence and tokenize it 
random_scentence = random.choice(train_scentences)
print(f'original text {random_scentence} \n {text_vectorizer([random_scentence])}')


original text New Ladies Shoulder Tote Handbag Women Cross Body Bag Faux Leather Fashion Purse - Full reÛ_ http://t.co/BLAAWHYScT http://t.co/dDR0zjXVQN 
 [[  50  814  530  980 1014  427  248   83  322 1139  703 1024 1183  120
   644]]


In [26]:
# get the unique words in the vocab 
words_in_vocab = text_vectorizer.get_vocabulary() # getting all the unique words in training data 
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
print(f'no of words in vocab = {len(words_in_vocab)}')
print(f'top 5 words = {top_5_words}')
print(f'bottom 5 words = {bottom_5_words}')

no of words in vocab = 10000
top 5 words = ['', '[UNK]', 'the', 'a', 'in']
bottom 5 words = ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


In [27]:
## creating an embedding with embedding layer 
# that turning the length of the numbers which can be learnt during training and the output dimensions are fixed 
# the numbers in the embedding are started random but they are adjust over all training 


the paraameters include 
* `input_dims` : size of the vocab 
* `output_dims` : size of the output shape
* `input_length`: length of sequences being passed to embedding layer 

In [28]:
from tensorflow.keras import layers 
embedding = layers.Embedding(input_dim = max_vocab_length,
                             output_dim = 128,
                             input_length = max_length
                             )

In [29]:
embedding

In [30]:
random_scentence = random.choice(train_scentences)
print(f'original ext : {random_scentence}\n \n Embedding version')

# embedding the random scenetence (turning it to the text vectorizer )
sample_embed = embedding(text_vectorizer([random_scentence]))
sample_embed

original ext : #YIAYplan Use my awesome collection of Amiibos to destroy all in my path.
 
 Embedding version


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.02096686,  0.00787168, -0.04643998, ...,  0.00262858,
          0.01808388,  0.02912438],
        [-0.02084119, -0.01165282, -0.03098552, ...,  0.04275597,
         -0.0321843 , -0.03010612],
        [-0.04601506, -0.00581674,  0.04632136, ..., -0.00622628,
         -0.00206596,  0.04097344],
        ...,
        [ 0.02297249, -0.03861836, -0.04987041, ...,  0.02710168,
          0.00132129,  0.00208322],
        [-0.00929687, -0.016079  , -0.00511848, ...,  0.00931124,
         -0.04245986,  0.0433519 ],
        [-0.00929687, -0.016079  , -0.00511848, ...,  0.00931124,
         -0.04245986,  0.0433519 ]]], dtype=float32)>

In [31]:
# checking out the single token embedding 
sample_embed[0][0], sample_embed[0][0].shape,sample_embed[0]

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 0.02096686,  0.00787168, -0.04643998,  0.01251506,  0.0061677 ,
        -0.01907527,  0.04526741,  0.0398928 ,  0.02304802,  0.0412532 ,
        -0.0195897 ,  0.04751955,  0.01986944, -0.00809786, -0.01128526,
         0.02438966, -0.03365203,  0.04132574, -0.04581469, -0.00577153,
        -0.03247614,  0.0018013 , -0.01567619, -0.03911495, -0.00693326,
         0.02196108,  0.04375518,  0.04021765,  0.00457884, -0.03934147,
        -0.01909686, -0.03520172,  0.02622889,  0.03911335, -0.04372568,
        -0.0354808 , -0.00769919, -0.02506034,  0.01647276, -0.0343917 ,
        -0.04224335, -0.02939383,  0.02358517,  0.03682509,  0.02903653,
         0.01802022,  0.02379321,  0.04835639,  0.01569004, -0.03647665,
        -0.04238136,  0.0470794 , -0.0244767 , -0.03011832,  0.00550187,
         0.02261114,  0.03588294,  0.035131  , -0.03243115, -0.02384651,
        -0.00434095, -0.04948161,  0.03929475, -0.01526491, -0.04535766,
  

0. Naive base model with the tf idf encoder 
1. feed forward neural network (dense model)
2. LSTM (RNN)
3. GRU (RNN)
4. bidirectoinal LSTM (RNN)
5. 1D convolution neural network 
6. tf hub pretrained feature extracter 
7. tf hub pretrained feature extractor 10 % data 

## Modelling the text dataset (running the series of experimetns)

now we've a got a way to turn our text sequences into the numbers it's time to start for the modelling experiments

How are we going to approach 

standard steps in modelling with the tensorflow 
* create 
* build 
* fit 
* evaluate

base line model sklearn's Naive based model cause there speed and later using dl for improving upon them

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline


model_0 = Pipeline([
                    ('tfidf',TfidfVectorizer()),
                    ('clf',MultinomialNB()) # model the text 
])

# fitting the pipeline to the training data 
model_0.fit(train_scentences,train_labels)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [33]:
#Evaluating the baseline model 
baseline_score = model_0.score(val_scentences,val_labels)

In [34]:
baseline_score

0.7926509186351706

In [35]:
baseline_preds = model_0.predict(val_scentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [36]:
base_path = '/content/drive/Shareddrives/Tabata/NLP_TF'
# model_0.save(str(base_path+'/model_0.h5'))


In [37]:
#creating the evaluation function that will give the various evaluation matrix functiona 

we could evelauate the all of the model prediction with the different types but it will be combursome and easily be fixed with the function

* accuracy 
* precision
* recall
* f1 score 


In [38]:
from sklearn.metrics import accuracy_score,precision_recall_fscore_support
def create_evalute_func(y_true,y_preds):
  """
  calculate the model accuracy precision recall and f1 score of binary classification mode  """
  model_accuracy = accuracy_score(y_true,y_preds) * 100 
  # calculating the model precision recall wih tthe f1 score with the weighted average 
  model_precision , model_recall , model_f1 ,support = precision_recall_fscore_support(y_true,y_preds)
  
  print(f'accuracy is {model_accuracy} \n model_precision = {model_precision} \n model_recall= {model_recall} \n model_f1  = {model_f1} \n model_support = {support}')
  return model_accuracy,model_precision,model_recall,model_f1,support



In [39]:
create_evalute_func(val_labels , baseline_preds)

accuracy is 79.26509186351706 
 model_precision = [0.74806202 0.88617886] 
 model_recall= [0.93236715 0.62643678] 
 model_f1  = [0.83010753 0.73400673] 
 model_support = [414 348]


(79.26509186351706,
 array([0.74806202, 0.88617886]),
 array([0.93236715, 0.62643678]),
 array([0.83010753, 0.73400673]),
 array([414, 348]))

In [40]:
# feed forward neural network dense model 
# creating the tensorboard callback :

from helper_functions import create_tensorboard_callback

SAVE_DIR = 'model_logs'



In [41]:
# building the model wiht the functional api 
from tensorflow.keras import layers 
inputs = layers.Input(shape = (1,),dtype = tf.string) # inputs are the 1 d will error if we pass the non string like inputs 
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1,activation = 'sigmoid')(x)
model_1 = tf.keras.Model(inputs,outputs,name = 'model_1') 


In [42]:
model_1.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
global_average_pooling1d (Gl (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 1,280,129
Trainable params: 1,280,129
Non-trainable params: 0
_________________________________________________________________


In [43]:
# compiling the model 
model_1.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

In [44]:
model_1_history = model_1.fit(train_scentences, train_labels,
                              epochs =5 , 
                              validation_data = (val_scentences,val_labels),
                              callbacks =[create_tensorboard_callback(SAVE_DIR,'model_1_dense')])

Saving TensorBoard log files to: model_logs/model_1_dense/20210613-104015
Epoch 1/5
215/215 [==============================] - 6s 16ms/step - loss: 0.6081 - accuracy: 0.6961 - val_loss: 0.5323 - val_accuracy: 0.7572
Epoch 2/5
215/215 [==============================] - 3s 13ms/step - loss: 0.4392 - accuracy: 0.8171 - val_loss: 0.4656 - val_accuracy: 0.7848
Epoch 3/5
215/215 [==============================] - 3s 13ms/step - loss: 0.3466 - accuracy: 0.8640 - val_loss: 0.4579 - val_accuracy: 0.7874
Epoch 4/5
215/215 [==============================] - 3s 13ms/step - loss: 0.2835 - accuracy: 0.8930 - val_loss: 0.4661 - val_accuracy: 0.7887
Epoch 5/5
215/215 [==============================] - 3s 13ms/step - loss: 0.2377 - accuracy: 0.9108 - val_loss: 0.4823 - val_accuracy: 0.7887


In [45]:
model_1.evaluate(val_scentences,val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.4823 - accuracy: 0.7887


[0.48230281472206116, 0.7887139320373535]

In [46]:
# making some prediction and evaluate those 
model_1_pred_probs = model_1.predict(val_scentences)
model_1_pred_probs,model_1_pred_probs.shape

(array([[3.38533700e-01],
        [7.47287571e-01],
        [9.97851849e-01],
        [1.20838284e-01],
        [1.07824363e-01],
        [9.35357511e-01],
        [8.77429783e-01],
        [9.93329585e-01],
        [9.55659807e-01],
        [2.97393531e-01],
        [1.08337902e-01],
        [6.23547316e-01],
        [3.84020731e-02],
        [1.58325106e-01],
        [4.38441150e-03],
        [1.24097124e-01],
        [2.21764874e-02],
        [8.88254642e-02],
        [2.23319292e-01],
        [4.73034799e-01],
        [8.96429777e-01],
        [3.98857072e-02],
        [3.87118191e-01],
        [7.62801990e-02],
        [9.56083536e-01],
        [9.98526037e-01],
        [3.54289860e-02],
        [6.23492226e-02],
        [2.20651664e-02],
        [1.86621636e-01],
        [5.18037200e-01],
        [2.57401377e-01],
        [4.44233656e-01],
        [1.60661161e-01],
        [4.28556830e-01],
        [6.67440817e-02],
        [9.94037867e-01],
        [1.54364929e-01],
        [3.4

In [47]:
model_1_pred_probs[:1]

array([[0.3385337]], dtype=float32)

In [48]:
model_1_pred_probs[:10]

array([[0.3385337 ],
       [0.7472876 ],
       [0.99785185],
       [0.12083828],
       [0.10782436],
       [0.9353575 ],
       [0.8774298 ],
       [0.9933296 ],
       [0.9556598 ],
       [0.29739353]], dtype=float32)

In [49]:
create_evalute_func(val_labels,tf.round(tf.squeeze(model_1_pred_probs)))

accuracy is 78.87139107611549 
 model_precision = [0.75975359 0.84      ] 
 model_recall= [0.89371981 0.6637931 ] 
 model_f1  = [0.82130966 0.74157303] 
 model_support = [414 348]


(78.87139107611549,
 array([0.75975359, 0.84      ]),
 array([0.89371981, 0.6637931 ]),
 array([0.82130966, 0.74157303]),
 array([414, 348]))

In [50]:
# model_1.save('/drive/MyDrive/SOM_models/NLP_dense')

INFO:tensorflow:Assets written to: /drive/MyDrive/SOM_models/NLP_dense/assets


In [51]:
# not all the experiments which we run may not be the improvement over another models  /drive/MyDrive/SOM_models/NLP_dense/assets


# visualizing the embedding layer 

( embedding is the numerical representation of all the data in the dataset (words) ) 



In [ ]:
# getting the vocab from text vectorization layer 
words_in_vocab = text_vectorizer.get_vocabulary()

len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [ ]:
max_vocab_length

10000

In [ ]:
model_1.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
global_average_pooling1d (Gl (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 1,280,129
Trainable params: 1,280,129
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#inspecting the embedding layer 

In [ ]:
# getting the weight matrix of the embedding layer 
# these are the numerical representation of each token in the training  data which is there for 5 epochs 
embed_weighs = model_1.get_layer('embedding').get_weights()[0]

In [ ]:
print(embed_weighs)

[[-0.05818793  0.03887457 -0.01769985 ... -0.03974245  0.0186508
   0.03563593]
 [-0.05294124  0.05990492 -0.0571956  ... -0.01517506  0.02008834
  -0.02023306]
 [ 0.01806652  0.00702938 -0.0529419  ... -0.02176655 -0.05447836
   0.03719018]
 ...
 [ 0.0346168  -0.03618568 -0.01617943 ...  0.01820603  0.02265736
   0.00552472]
 [-0.1169711   0.08146672  0.06968097 ... -0.04372403  0.1081845
   0.06406015]
 [-0.1542515   0.14224572  0.06646438 ... -0.15120101  0.09387157
   0.13091907]]


In [ ]:
# visualizing the embedding weights 
embed_weighs.shape

(10000, 128)

tensorflow has the handy tool called projector so that we can plot the projection https://projector.tensorflow.org

tensorflow word embeddings has the great documentation too


In [ ]:
import io 
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weighs[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [ ]:
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

  ## Recurrent Neural Networks 

  RNN are useful for the sequence data 

  the preise of the rnn is to use teh representation of the previous input to aid the representation of the later input 

#2 LSTM RNN


## Model_2: LSTM 
LSTM = long short term memory this is one of the most popular lstm cell 
our structure of rnn is like 

``` 
Input ( text ) ----> tokenize ------> Embedding --------> Layers (RNN/Dense) -------> Output(label probabilty)
```

In [ ]:
from tensorflow.keras import layers

inputs = layers.Input(shape = (1,),dtype = 'string')
x = text_vectorizer(inputs)

print(x.shape)
x = embedding(x)
print(x.shape)
x = layers.LSTM(64,return_sequences= True)(x) #when we are stakinc the RNN sequnces we need to set return sequences as True 
print(x.shape)
x = layers.LSTM(64)(x)
print(x.shape)
x = layers.Dense(64,activation = 'relu')(x)
outputs = layers.Dense(1,activation = 'sigmoid')(x)
model_2 = tf.keras.Model(inputs,outputs,name = 'model_2_LSTM')

(None, 15)
(None, 15, 128)
(None, 15, 64)
(None, 64)


In [ ]:
model_2.summary()

In [ ]:
# compiling the model 
model_2.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

In [ ]:
model_2_history = model_2.fit(train_scentences,train_labels,
                              epochs = 5 , 
                              validation_data = (val_scentences,val_labels),
                              callbacks = [create_tensorboard_callback(SAVE_DIR,'model_2_LSTM')])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20210611-070423
Epoch 1/5
215/215 [==============================] - 9s 27ms/step - loss: 0.0430 - accuracy: 0.9809 - val_loss: 1.1433 - val_accuracy: 0.7651
Epoch 2/5
215/215 [==============================] - 4s 20ms/step - loss: 0.0348 - accuracy: 0.9834 - val_loss: 1.6928 - val_accuracy: 0.7598
Epoch 3/5
215/215 [==============================] - 4s 20ms/step - loss: 0.0403 - accuracy: 0.9819 - val_loss: 1.3187 - val_accuracy: 0.7677
Epoch 4/5
215/215 [==============================] - 4s 20ms/step - loss: 0.0363 - accuracy: 0.9823 - val_loss: 1.6160 - val_accuracy: 0.7638
Epoch 5/5
215/215 [==============================] - 4s 19ms/step - loss: 0.0405 - accuracy: 0.9825 - val_loss: 1.4196 - val_accuracy: 0.7690


In [ ]:
# making the predictions with the lstm model 
model_2_pred_probs = model_2.predict(val_scentences)

In [ ]:
model_2_pred_probs[:10]

array([[1.2962222e-03],
       [8.2363647e-01],
       [9.9999332e-01],
       [8.3877079e-02],
       [1.0931251e-05],
       [9.9985445e-01],
       [9.3936938e-01],
       [9.9999297e-01],
       [9.9999189e-01],
       [9.9416912e-01]], dtype=float32)

In [ ]:
model_2.save('/content/drive/MyDrive/SOM_models/model_2_LSTM2LAYERRNN.h5')

NotImplementedError: ignored

In [ ]:
create_evalute_func(val_labels,tf.round(tf.squeeze(model_2_pred_probs)))

accuracy is 76.37795275590551 
 model_precision = [0.75770925 0.77272727] 
 model_recall= [0.83091787 0.68390805] 
 model_f1  = [0.79262673 0.72560976] 
 model_support = [414 348]


(76.37795275590551,
 array([0.75770925, 0.77272727]),
 array([0.83091787, 0.68390805]),
 array([0.79262673, 0.72560976]),
 array([414, 348]))

In [ ]:
loaded_model = tf.keras.models.load_model('/content/drive/MyDrive/SOM_models/model_2_LSTM2LAYERRNN')

In [ ]:
loaded_model.evaluate(val_scentences,val_labels)

24/24 [==============================] - 1s 5ms/step - loss: 1.6751 - accuracy: 0.7638


[1.6751083135604858, 0.7637795209884644]

In [ ]:
# building the model with the GRU (Gated Recurrent Unit)
# the gru cell has the similar features to the lstm cell but have the less parameter LSST

inputs = tf.keras.layers.Input(shape =(1,),dtype = 'string')
x = text_vectorizer(inputs)
x = embedding(x)
x = tf.keras.layers.GRU(64,return_sequences=True)(x)
# x = tf.keras.layers.LSTM(64,return_sequences=True)(x)
# x = tf.keras.layers.GRU(64)(x)
x = tf.keras.layers.GRU(64)(x)
# x = tf.keras.layers.GlobalAveragePooling1D()(x)
outputs = tf.keras.layers.Dense(1,activation = 'sigmoid')(x)
model_3 = tf.keras.Model(inputs,outputs, name = "GRU")

In [ ]:
model_3.summary()

Model: "GRU"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
gru_10 (GRU)                 (None, 15, 64)            37248     
_________________________________________________________________
gru_11 (GRU)                 (None, 64)                24960     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65        
Total params: 1,342,273
Trainable params: 1,342,273
Non-trainable params: 0
_____________________________________________________

In [ ]:
model_3.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

In [ ]:
history_model_3 = model_3.fit(train_scentences,train_labels,
                              epochs = 5 , 
                              validation_data = (val_scentences,val_labels),
                              callbacks = [create_tensorboard_callback(SAVE_DIR,'GRU_MODEL')])

Saving TensorBoard log files to: model_logs/GRU_MODEL/20210611-131324
Epoch 1/5
215/215 [==============================] - 5s 21ms/step - loss: 0.0402 - accuracy: 0.9807 - val_loss: 1.4824 - val_accuracy: 0.7480
Epoch 2/5
215/215 [==============================] - 4s 18ms/step - loss: 0.0381 - accuracy: 0.9799 - val_loss: 1.5538 - val_accuracy: 0.7572
Epoch 3/5
215/215 [==============================] - 4s 18ms/step - loss: 0.0382 - accuracy: 0.9820 - val_loss: 1.2244 - val_accuracy: 0.7572
Epoch 4/5
215/215 [==============================] - 4s 18ms/step - loss: 0.0365 - accuracy: 0.9804 - val_loss: 1.6328 - val_accuracy: 0.7546
Epoch 5/5
215/215 [==============================] - 4s 18ms/step - loss: 0.0445 - accuracy: 0.9801 - val_loss: 1.3901 - val_accuracy: 0.7572


In [ ]:
model_3.predict(val_scentences)

array([[2.89068237e-04],
       [5.80871344e-01],
       [9.98812675e-01],
       [1.44893378e-01],
       [1.41820245e-04],
       [9.99109089e-01],
       [4.89267558e-01],
       [9.99674916e-01],
       [9.99592960e-01],
       [8.19597006e-01],
       [1.52143824e-04],
       [9.26533461e-01],
       [3.66180553e-04],
       [1.60296604e-01],
       [9.86484592e-05],
       [5.22368180e-04],
       [2.13405787e-04],
       [1.80767631e-04],
       [4.57693310e-03],
       [9.89741445e-01],
       [9.95396316e-01],
       [9.62329141e-05],
       [9.98157442e-01],
       [2.78269086e-04],
       [9.99631047e-01],
       [9.99671578e-01],
       [2.43254413e-04],
       [3.70947178e-04],
       [6.12140226e-04],
       [9.81391847e-01],
       [9.83616233e-01],
       [1.59263218e-04],
       [1.92985300e-03],
       [4.24667320e-04],
       [2.30841368e-01],
       [8.31000209e-01],
       [9.99391437e-01],
       [6.74799457e-02],
       [6.69698871e-04],
       [9.99574006e-01],


In [ ]:
model_3.save('/content/drive/MyDrive/SOM_models/GRU_MODEL1')

NotImplementedError: ignored

In [ ]:
create_evalute_func(val_labels, tf.round(tf.squeeze(model_3.predict(val_scentences))))

accuracy is 75.7217847769029 
 model_precision = [0.73706004 0.7921147 ] 
 model_recall= [0.85990338 0.63505747] 
 model_f1  = [0.79375697 0.70494418] 
 model_support = [414 348]


(75.7217847769029,
 array([0.73706004, 0.7921147 ]),
 array([0.85990338, 0.63505747]),
 array([0.79375697, 0.70494418]),
 array([414, 348]))

> normal vs bidirectional lstms 
* normal ======> left to right 
* bidirectional ========> right to left as well as left to right 

In [ ]:
inputs = tf.keras.layers.Input(shape = (1,),dtype ='string')
x = text_vectorizer(inputs)
x = embedding(x)
# x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,return_sequences= True))(x)
# x = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64,return_sequences=True))(x)
x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64))(x)
outputs = tf.keras.layers.Dense(1,activation ='sigmoid')(x)


model_4 = tf.keras.Model(inputs,outputs, name = 'bidirectionnal_model')

In [ ]:
model_4.compile(loss = tf.keras.losses.binary_crossentropy,
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])


In [ ]:
model_4_history = model_4.fit(train_scentences,train_labels,
                              epochs = 5 , 
                              validation_data = (val_scentences,val_labels),
                              callbacks = [create_tensorboard_callback(SAVE_DIR,'bidirectional_model')])

Saving TensorBoard log files to: model_logs/bidirectional_model/20210611-134421
Epoch 1/5
215/215 [==============================] - 9s 28ms/step - loss: 0.1004 - accuracy: 0.9650 - val_loss: 1.3335 - val_accuracy: 0.7454
Epoch 2/5
215/215 [==============================] - 5s 21ms/step - loss: 0.0370 - accuracy: 0.9813 - val_loss: 1.3944 - val_accuracy: 0.7546
Epoch 3/5
215/215 [==============================] - 5s 21ms/step - loss: 0.0380 - accuracy: 0.9810 - val_loss: 1.5141 - val_accuracy: 0.7467
Epoch 4/5
215/215 [==============================] - 5s 21ms/step - loss: 0.0389 - accuracy: 0.9818 - val_loss: 1.2355 - val_accuracy: 0.7480
Epoch 5/5
215/215 [==============================] - 5s 21ms/step - loss: 0.0341 - accuracy: 0.9839 - val_loss: 1.5041 - val_accuracy: 0.7428


In [ ]:
model_4.evaluate(val_scentences,val_labels)

24/24 [==============================] - 0s 4ms/step - loss: 1.5041 - accuracy: 0.7428


[1.5041486024856567, 0.7427821755409241]

In [ ]:
model_4.save('/content/drive/MyDrive/SOM_models/bidirectional_02')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/SOM_models/bidirectional_02/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/SOM_models/bidirectional_02/assets


In [ ]:
loaded_bidirectional = tf.keras.models.load_model('/content/drive/MyDrive/SOM_models/bidirectional_02')

In [ ]:
loaded_bidirectional.evaluate(val_scentences,val_labels)

24/24 [==============================] - 1s 5ms/step - loss: 1.5041 - accuracy: 0.7428


[1.5041486024856567, 0.7427821755409241]

In [ ]:
loaded_bidirectional.summary()

Model: "bidirectionnal_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
bidirectional_11 (Bidirectio (None, 128)               98816     
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 129       
Total params: 1,378,945
Trainable params: 1,378,945
Non-trainable params: 0
_________________________________________________________________


In [ ]:
create_evalute_func(val_labels, tf.round(tf.squeeze(loaded_bidirectional.predict(val_scentences))))

accuracy is 74.2782152230971 
 model_precision = [0.7546729  0.72754491] 
 model_recall= [0.78019324 0.69827586] 
 model_f1  = [0.7672209  0.71260997] 
 model_support = [414 348]


(74.2782152230971,
 array([0.7546729 , 0.72754491]),
 array([0.78019324, 0.69827586]),
 array([0.7672209 , 0.71260997]),
 array([414, 348]))

In [51]:
#1D convolutional neural netowork 
inputs = tf.keras.layers.Input(shape = (1,),dtype = 'string')
x = text_vectorizer(inputs)
x = embedding(x)
x = tf.keras.layers.Conv1D(64,4,1,activation='relu')(x) # padding = valid (default) the output is smaller than the padding wihh the same 
x = tf.keras.layers.GlobalMaxPool1D()(x)
outputs = tf.keras.layers.Dense(1,activation ='sigmoid')(x)
model_5 = tf.keras.Model(inputs, outputs)

In [52]:
model_5.compile(loss = tf.keras.losses.binary_crossentropy,
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

In [53]:
model_5_history = model_5.fit(train_scentences,train_labels,
                              epochs = 5 , 
                              validation_data = (val_scentences,val_labels),
                              callbacks = [create_tensorboard_callback(SAVE_DIR,'1D_CONV')])

Saving TensorBoard log files to: model_logs/1D_CONV/20210613-043502
Epoch 1/5
215/215 [==============================] - 19s 17ms/step - loss: 0.5497 - accuracy: 0.7272 - val_loss: 0.4649 - val_accuracy: 0.7874
Epoch 2/5
215/215 [==============================] - 3s 14ms/step - loss: 0.3364 - accuracy: 0.8653 - val_loss: 0.4734 - val_accuracy: 0.7848
Epoch 3/5
215/215 [==============================] - 3s 14ms/step - loss: 0.2072 - accuracy: 0.9245 - val_loss: 0.5477 - val_accuracy: 0.7887
Epoch 4/5
215/215 [==============================] - 3s 14ms/step - loss: 0.1266 - accuracy: 0.9591 - val_loss: 0.6384 - val_accuracy: 0.7782
Epoch 5/5
215/215 [==============================] - 3s 14ms/step - loss: 0.0900 - accuracy: 0.9707 - val_loss: 0.6817 - val_accuracy: 0.7703


In [54]:
model_5.evaluate(val_scentences,val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.6817 - accuracy: 0.7703


[0.6817424893379211, 0.7703412175178528]

In [55]:
model_5.save('/content/drive/MyDrive/SOM_models/1D_Conv')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/SOM_models/1D_Conv/assets


In [56]:
conv_load = tf.keras.models.load_model('/content/drive/MyDrive/SOM_models/1D_Conv')

In [57]:
conv_load.evaluate(val_scentences,val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.6817 - accuracy: 0.7703


[0.6817424893379211, 0.7703412175178528]

In [58]:
print('helo')

helo


## Model 5 conv1D 
padding : same = preserves all the values add's zeros to the values if the filter size is greater than the numbers which are available in the input layer 

padding = valid : does not preserves all the values it discards values when the filter size is greater than the number available in the sequence 


In [60]:
conv_1d_outuput = conv_load.predict(val_scentences)

In [61]:
conv_1d_outuput

array([[5.92199624e-01],
       [1.45118445e-01],
       [9.99777853e-01],
       [4.44535278e-02],
       [3.20257922e-03],
       [9.96819139e-01],
       [9.74591792e-01],
       [9.98735487e-01],
       [9.99525785e-01],
       [1.08942568e-01],
       [1.23625221e-02],
       [7.41489053e-01],
       [9.35160927e-03],
       [3.71117532e-01],
       [1.34503862e-04],
       [1.75149720e-02],
       [1.08385207e-02],
       [4.00968820e-01],
       [4.70376872e-02],
       [4.18284923e-01],
       [6.00339592e-01],
       [1.04552484e-03],
       [8.33274841e-01],
       [2.48887148e-02],
       [9.97891486e-01],
       [9.99970675e-01],
       [1.77183226e-02],
       [3.56139336e-03],
       [8.53877794e-03],
       [3.52752566e-01],
       [9.69432056e-01],
       [1.47566900e-01],
       [5.80294609e-01],
       [7.72084653e-01],
       [9.80988920e-01],
       [7.83611774e-01],
       [9.99911189e-01],
       [4.66336198e-02],
       [8.62022024e-03],
       [9.99813497e-01],


In [63]:
create_evalute_func(val_labels, tf.squeeze(tf.round(conv_load.predict(val_scentences))))

accuracy is 77.03412073490814 
 model_precision = [0.76614699 0.77635783] 
 model_recall= [0.83091787 0.69827586] 
 model_f1  = [0.797219   0.73524962] 
 model_support = [414 348]


(77.03412073490814,
 array([0.76614699, 0.77635783]),
 array([0.83091787, 0.69827586]),
 array([0.797219  , 0.73524962]),
 array([414, 348]))

# tfhub pretrained scentence encoder ,

In [214]:
import tensorflow_hub as tf_hub 
base_model_url = 'https://tfhub.dev/google/universal-sentence-encoder/4'
embed = tf_hub.load(base_model_url)
# base_model.trainble = False

In [215]:
embed_sample = embed(['this is the sample scentence' , 'when you go univerassal scentence encoder to the scentence it turn it into the number sample '])
embed_sample


<tf.Tensor: shape=(2, 512), dtype=float32, numpy=
array([[-0.00546509, -0.01532443,  0.04782044, ..., -0.03626173,
        -0.00051198,  0.05656442],
       [ 0.00648542, -0.07160369, -0.00667657, ..., -0.04834846,
         0.03524181,  0.02192586]], dtype=float32)>

In [216]:
scentence_encoder_layer = tf_hub.KerasLayer(base_model_url,input_shape = [],
                                             dtype = tf.string , 
                                             trainable = False,
                                             name = 'USE')

In [217]:
# creating the model using sequential api 
model_6 = tf.keras.Sequential([
                               scentence_encoder_layer,
                               tf.keras.layers.Dense(1,activation= 'sigmoid',name = 'output_layer'),
])

In [218]:
model_6.compile(loss = tf.keras.losses.binary_crossentropy,
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

In [219]:
model_6.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
USE (KerasLayer)             (None, 512)               256797824 
_________________________________________________________________
output_layer (Dense)         (None, 1)                 513       
Total params: 256,798,337
Trainable params: 513
Non-trainable params: 256,797,824
_________________________________________________________________


In [220]:
model_6_history = model_6.fit(train_scentences,train_labels,
            epochs = 5 , 
            callbacks =[create_tensorboard_callback(SAVE_DIR,'transfer_learning_model_1')],
            validation_data = (val_scentences,val_labels),
            )

Saving TensorBoard log files to: model_logs/transfer_learning_model_1/20210613-134755
Epoch 1/5
215/215 [==============================] - 8s 30ms/step - loss: 0.6507 - accuracy: 0.7438 - val_loss: 0.6138 - val_accuracy: 0.7940
Epoch 2/5
215/215 [==============================] - 2s 10ms/step - loss: 0.5829 - accuracy: 0.7942 - val_loss: 0.5636 - val_accuracy: 0.7887
Epoch 3/5
215/215 [==============================] - 2s 10ms/step - loss: 0.5394 - accuracy: 0.7994 - val_loss: 0.5314 - val_accuracy: 0.7913
Epoch 4/5
215/215 [==============================] - 2s 9ms/step - loss: 0.5104 - accuracy: 0.7981 - val_loss: 0.5103 - val_accuracy: 0.7900
Epoch 5/5
215/215 [==============================] - 2s 10ms/step - loss: 0.4900 - accuracy: 0.8003 - val_loss: 0.4957 - val_accuracy: 0.7913


In [221]:
# model_6.save('/content/drive/MyDrive/SOM_models/transfer_learning_1')

In [222]:
create_evalute_func(val_labels, tf.squeeze(tf.round(model_6.predict(val_scentences))))

accuracy is 79.13385826771653 
 model_precision = [0.78396437 0.80191693] 
 model_recall= [0.85024155 0.72126437] 
 model_f1  = [0.81575898 0.75945537] 
 model_support = [414 348]


(79.13385826771653,
 array([0.78396437, 0.80191693]),
 array([0.85024155, 0.72126437]),
 array([0.81575898, 0.75945537]),
 array([414, 348]))

In [223]:
model_7 = tf.keras.Sequential([
                               scentence_encoder_layer,
                               tf.keras.layers.Dense(64 , activation = 'relu', name = 'dense_layer'),
                               tf.keras.layers.Dense(1,activation = 'sigmoid' , name = 'output_layer')
])

In [224]:
model_7.compile(
    loss = tf.keras.losses.binary_crossentropy,
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ['accuracy']
)

In [225]:
model_7_history = model_7.fit(
    train_scentences,train_labels,
    epochs =5 , 
    callbacks = [create_tensorboard_callback(SAVE_DIR,'transfer_2_dense')],
    validation_data = (val_scentences,val_labels)
)

Saving TensorBoard log files to: model_logs/transfer_2_dense/20210613-134818
Epoch 1/5
215/215 [==============================] - 4s 16ms/step - loss: 0.5107 - accuracy: 0.7815 - val_loss: 0.4475 - val_accuracy: 0.7953
Epoch 2/5
215/215 [==============================] - 2s 10ms/step - loss: 0.4145 - accuracy: 0.8170 - val_loss: 0.4364 - val_accuracy: 0.8045
Epoch 3/5
215/215 [==============================] - 2s 10ms/step - loss: 0.4014 - accuracy: 0.8228 - val_loss: 0.4313 - val_accuracy: 0.8136
Epoch 4/5
215/215 [==============================] - 2s 9ms/step - loss: 0.3932 - accuracy: 0.8244 - val_loss: 0.4283 - val_accuracy: 0.8176
Epoch 5/5
215/215 [==============================] - 2s 9ms/step - loss: 0.3848 - accuracy: 0.8320 - val_loss: 0.4248 - val_accuracy: 0.8163


In [226]:
# model_7.save('/content/drive/MyDrive/SOM_models/transfer_2_dense_81_per')

In [227]:
load_model_7 = tf.keras.models.load_model('/content/drive/MyDrive/SOM_models/transfer_2_dense_81_per')

In [228]:
load_model_7.evaluate(val_scentences,val_labels)

24/24 [==============================] - 1s 8ms/step - loss: 0.4287 - accuracy: 0.8176


[0.4287063181400299, 0.817585289478302]

In [229]:
# splitting the data as 10 % of the data 
from sklearn.model_selection import train_test_split

In [230]:
# training the model_7 on the 10 % of the training data 

In [231]:
# creating the subsets 
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [232]:

# Note : making the data splits like below leads to the data leakage model validation data is from the same dataset as teh  
# train_10_per = train_df_shuffled[['text','target']].sample(frac = 0.1,random_state = 42)

# train_scentences_10_per = train_10_per['text'].to_list()
# train_labels_10_per = train_10_per['target'].to_list()

In [233]:
train_10_per.head()

NameError: ignored

In [ ]:
train_10_per

In [ ]:
len(train_scentences_10_per)

In [ ]:
train_10_per['target'].value_counts()

In [234]:
model_8 = tf.keras.models.clone_model(model_7)

In [235]:
model_8.compile(
    loss = tf.keras.losses.binary_crossentropy,
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ['accuracy']
)

In [236]:
another_model_10_per = model_8.fit(
    train_scentences_10_per,train_labels_10_per,
    epochs = 5 , 
    validation_data = (val_scentences,val_labels),
    callbacks = [create_tensorboard_callback(SAVE_DIR,'10_per')]
)

NameError: ignored

In [121]:
model_8.save('/content/drive/MyDrive/SOM_models/transfer_learning_10_per_83_latest')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/SOM_models/transfer_learning_10_per_83_latest/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/SOM_models/transfer_learning_10_per_83_latest/assets


In [58]:
loaded_model_8 = tf.keras.models.load_model('/content/drive/MyDrive/SOM_models/transfer_learning_10_per_83_latest')

In [59]:
loaded_model_8.evaluate(val_scentences, val_labels)

24/24 [==============================] - 1s 9ms/step - loss: 0.4064 - accuracy: 0.8307


[0.4063825309276581, 0.8307086825370789]

In [60]:
%load_ext tensorboard 
!tensorboard dev upload --logdir model_logs --one_shot

2021-06-13 10:44:51.102180: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0

***** TensorBoard Uploader *****

This will upload your TensorBoard logs to https://tensorboard.dev/ from
the following directory:

model_logs

This TensorBoard will be visible to everyone. Do not upload sensitive
data.

Your use of this service is subject to Google's Terms of Service
<https://policies.google.com/terms> and Privacy Policy
<https://policies.google.com/privacy>, and TensorBoard.dev's Terms of Service
<https://tensorboard.dev/policy/terms/>.

This notice will not be shown again while you are logged into the uploader.
To log out, run `tensorboard dev auth revoke`.

Continue? (yes/NO) Traceback (most recent call last):
  File "/usr/local/bin/tensorboard", line 8, in <module>
    sys.exit(run_main())
  File "/usr/local/lib/python3.7/dist-packages/tensorboard/main.py", line 46, in run_main
    app.run(tensorboard.main, flags_parser

In [61]:
train_10_percent_split = int(0.1 * len(train_scentences))
train_10_percent = train_scentences[:train_10_percent_split]

In [62]:
train_labels_10_percnet = train_labels[:train_10_percent_split]

In [63]:
len(train_labels_10_percnet)

685

In [64]:
train_labels_10_percnet

array([0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0,
       1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1,
       0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0,
       1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0,
       1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0,

In [65]:
train_10_percent

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk',
       '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
       'destroy the free fandom honestly',
       'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
       '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
       'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt',
       'Lava Dragon Breeder! I just bred a 

In [66]:
pd.Series(np.array(train_labels_10_percnet)).value_counts()

0    406
1    279
dtype: int64

In [67]:
# checking the number of each label in updated training labels and scentenc


In [68]:
print('new exe')

new exe


In [69]:
print('all the models history and the logs are uploaded on the tensorboard')

all the models history and the logs are uploaded on the tensorboard


In [70]:
# loading model wihtthe custom hub layer (required with the h5 format ) 

# loaded_model = tf.keras.models.load_model(model_url, custom_object = {'KerasLayer':hub.KerasLayer})

In [71]:
!wget https://storage.googleapis.com/ztm_tf_course/08_model_6_USE_feature_extractor.zip

--2021-06-13 11:43:32--  https://storage.googleapis.com/ztm_tf_course/08_model_6_USE_feature_extractor.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.216.128, 173.194.217.128, 108.177.12.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.216.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 960779165 (916M) [application/zip]
Saving to: ‘08_model_6_USE_feature_extractor.zip’

08_model_6_USE_feat 100%[===================>] 916.27M   128MB/s    in 7.5s    

2021-06-13 11:43:39 (122 MB/s) - ‘08_model_6_USE_feature_extractor.zip’ saved [960779165/960779165]



In [72]:
!unzip 08_model_6_USE_feature_extractor.zip

Archive:  08_model_6_USE_feature_extractor.zip
   creating: 08_model_6_USE_feature_extractor/
   creating: 08_model_6_USE_feature_extractor/assets/
   creating: 08_model_6_USE_feature_extractor/variables/
  inflating: 08_model_6_USE_feature_extractor/variables/variables.data-00000-of-00001  
  inflating: 08_model_6_USE_feature_extractor/variables/variables.index  
  inflating: 08_model_6_USE_feature_extractor/saved_model.pb  


In [73]:
daniel_model = tf.keras.models.load_model('/content/08_model_6_USE_feature_extractor')

In [74]:
daniel_model.evaluate(val_scentences,val_labels)

24/24 [==============================] - 1s 8ms/step - loss: 0.4272 - accuracy: 0.8163


[0.42723122239112854, 0.8162729740142822]

In [85]:
# making the predictoins with the daniel's model 
daniel_preds = tf.squeeze(daniel_model.predict(val_scentences))
daniel_pred_labels = tf.squeeze(tf.round(daniel_model.predict(val_scentences)))

In [86]:
# creating the dataframe with the validation scentences and the best performing model prediction labels + probabilities 
val_df = pd.DataFrame({'text':val_scentences,
                       'target': val_labels,
                       'pred':daniel_pred_labels,
                       'pred_prob':daniel_preds})

In [87]:
val_df

,text,target,pred,pred_prob
0,DFR EP016 Monthly Meltdown - On Dnbheaven 2015...,0,0.0,0.159757
1,FedEx no longer to transport bioterror germs i...,0,1.0,0.747162
2,Gunmen kill four in El Salvador bus attack: Su...,1,1.0,0.988749
3,@camilacabello97 Internally and externally scr...,1,0.0,0.196229
4,Radiation emergency #preparedness starts with ...,1,1.0,0.707808
...,...,...,...,...
757,That's the ultimate road to destruction,0,0.0,0.137074
758,@SetZorah dad why dont you claim me that mean ...,0,0.0,0.095865
759,FedEx will no longer transport bioterror patho...,0,1.0,0.876982
760,Crack in the path where I wiped out this morni...,0,1.0,0.612631


In [88]:
# find the wrong predictions and sort by the prediction probabilities 
most_wrong = val_df[val_df['target'] != val_df['pred']].sort_values('pred_prob',ascending = False)

In [89]:
most_wrong

,text,target,pred,pred_prob
31,? High Skies - Burning Buildings ? http://t.co...,0,1.0,0.910196
759,FedEx will no longer transport bioterror patho...,0,1.0,0.876982
628,@noah_anyname That's where the concentration c...,0,1.0,0.852300
209,Ashes 2015: AustraliaÛªs collapse at Trent Br...,0,1.0,0.835454
251,@AshGhebranious civil rights continued in the ...,0,1.0,0.827213
...,...,...,...,...
411,@SoonerMagic_ I mean I'm a fan but I don't nee...,1,0.0,0.043918
233,I get to smoke my shit in peace,1,0.0,0.042087
38,Why are you deluged with low self-image? Take ...,1,0.0,0.038998
244,Reddit Will Now QuarantineÛ_ http://t.co/pkUA...,1,0.0,0.038949


`0` not disaster 
`1` disaster 


In [114]:
for _,text,target,pred,pred_prob in most_wrong.head(10).itertuples():
  print(text)
  print(f'target is {target}')
  print(f'prediction model did is {pred} with probability {pred_prob}')
  print('--------------')

? High Skies - Burning Buildings ? http://t.co/uVq41i3Kx2 #nowplaying
target is 0
prediction model did is 1.0 with probability 0.9101957678794861
--------------
FedEx will no longer transport bioterror pathogens in wake of anthrax lab mishaps http://t.co/lHpgxc4b8J
target is 0
prediction model did is 1.0 with probability 0.8769821524620056
--------------
@noah_anyname That's where the concentration camps and mass murder come in. 
 
EVERY. FUCKING. TIME.
target is 0
prediction model did is 1.0 with probability 0.8523000478744507
--------------
Ashes 2015: AustraliaÛªs collapse at Trent Bridge among worst in history: England bundled out Australia for 60 ... http://t.co/t5TrhjUAU0
target is 0
prediction model did is 1.0 with probability 0.8354544043540955
--------------
@AshGhebranious civil rights continued in the 60s. And what about trans-generational trauma? if anything we should listen to the Americans.
target is 0
prediction model did is 1.0 with probability 0.8272132873535156
-----

In [115]:
for _,text,target,pred,pred_prob in most_wrong.tail(10).itertuples():
  print(text)
  print(f'target is {target}')
  print(f'prediction model did is {pred} with probability {pred_prob}')
  print('--------------')

@DavidVonderhaar At least you were sincere ??
target is 1
prediction model did is 0.0 with probability 0.06730347871780396
--------------
@willienelson We need help! Horses will die!Please RT &amp; sign petition!Take a stand &amp; be a voice for them! #gilbert23 https://t.co/e8dl1lNCVu
target is 1
prediction model did is 0.0 with probability 0.05507579818367958
--------------
Lucas Duda is Ghost Rider. Not the Nic Cage version but an actual 'engulfed in flames' badass. #Mets
target is 1
prediction model did is 0.0 with probability 0.05460338667035103
--------------
going to redo my nails and watch behind the scenes of desolation of smaug ayyy
target is 1
prediction model did is 0.0 with probability 0.05459698289632797
--------------
You can never escape me. Bullets don't harm me. Nothing harms me. But I know pain. I know pain. Sometimes I share it. With someone like you.
target is 1
prediction model did is 0.0 with probability 0.04963728412985802
--------------
@SoonerMagic_ I mean I'm

In [117]:
test_data

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [118]:
test_scentences =  test_data['text'].to_list() 

In [119]:
test_scentences

['Just happened a terrible car crash',
 'Heard about #earthquake is different cities, stay safe everyone.',
 'there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all',
 'Apocalypse lighting. #Spokane #wildfires',
 'Typhoon Soudelor kills 28 in China and Taiwan',
 "We're shaking...It's an earthquake",
 "They'd probably still show more life than Arsenal did yesterday, eh? EH?",
 'Hey! How are you?',
 'What a nice hat?',
 'Fuck off!',
 "No I don't like cold!",
 "NOOOOOOOOO! Don't do that!",
 "No don't tell me that!",
 'What if?!',
 'Awesome!',
 "Birmingham Wholesale Market is ablaze BBC News - Fire breaks out at Birmingham's Wholesale Market http://t.co/irWqCEZWEU",
 '@sunkxssedharry will you wear shorts for race ablaze ?',
 '#PreviouslyOnDoyinTv: Toke Makinwa\x89Ûªs marriage crisis sets Nigerian Twitter ablaze... http://t.co/CMghxBa2XI',
 'Check these out: http://t.co/rOI2NSmEJJ http://t.co/3Tj8ZjiN21 http://t.co/YDUiXEfIpE http://t.co/LxTjc87KLS 

In [120]:
test_labels = daniel_model.predict(test_scentences)

In [237]:
test_labels = model_7.predict(test_scentences)

In [238]:
test_labels

array([[0.6513289 ],
       [0.8936042 ],
       [0.8741441 ],
       ...,
       [0.9418228 ],
       [0.5283538 ],
       [0.74224305]], dtype=float32)

In [239]:
len(test_labels)

3263

In [240]:
new_df = test_data['id'].append(pd.Series(tf.squeeze(test_labels)))

In [241]:
new_df = pd.DataFrame(test_data['id'])

In [242]:
new_df.insert(1,'target',tf.squeeze(tf.cast(tf.round(test_labels),dtype=tf.int32).numpy()))

In [243]:
new_df.head()

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1


In [244]:
new_df.to_csv('submission.csv',index = False)

In [185]:
pd.read_csv('sample_submission.csv')

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0
...,...,...
3258,10861,0
3259,10865,0
3260,10868,0
3261,10874,0


In [171]:
pd.Series(tf.squeeze(tf.round(test_labels)).numpy())

0       1.0
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
3258    1.0
3259    1.0
3260    1.0
3261    1.0
3262    1.0
Length: 3263, dtype: float32

# the speed / score tradeoff 


the baseline model is really fast since it's made with the scikit learn but the another transfer learning model takes some time to predict so there is always some kind of trade off for the models which we're training on 

In [209]:
import time 
def pred_timer(model,samples):
  start_time = time.perf_counter() # start time 
  model.predict(samples)
  end_time = time.perf_counter()
  total_time = end_time - start_time 
  time_per_pred = total_time / len(samples)
  return total_time, time_per_pred 

In [210]:
pred_timer(daniel_model,test_scentences)

(0.7492356080001628, 0.00022961557094703118)

In [211]:
pred_timer(model_0,test_scentences)

(0.05824116700023296, 1.7848963224098365e-05)

while building the models we've to determine that the speed is important or the accuracy is important 